In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
print(tf.__version__)

2.18.0


In [3]:
dataset_path = r"C:\Users\shrau\OneDrive\Desktop\Jupiter projects\Machine-learning-projects\Diabetic_Retinopathy\gaussian_filtered_images"

In [4]:
IMG_SIZE = (224,224)
BATCH_SIZE = 32

In [5]:
#data augmentation
train_datagen = ImageDataGenerator(
    rescale = 1./255, #normalize the pixel values
    rotation_range =40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.3,
    horizontal_flip = True,
    fill_mode='nearest',
    validation_split = 0.2
)

In [6]:
#load_training images
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size = IMG_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    subset ='training'
)

Found 2931 images belonging to 5 classes.


In [8]:
val_datagen = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2
)

In [9]:
#load_training images
val_generator = val_datagen.flow_from_directory(
    dataset_path,
    target_size = IMG_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    subset ='validation'
)

Found 731 images belonging to 5 classes.


In [9]:
batch = next(val_generator)
print("Validation Batch Length:", len(batch))  # Should be 2 (images, labels)
print("Validation Image Shape:", batch[0].shape)
print("Validation Label Shape:", batch[1].shape)


Validation Batch Length: 2
Validation Image Shape: (32, 224, 224, 3)
Validation Label Shape: (32, 5)


In [10]:
print("Class indices: ", train_generator.class_indices)


Class indices:  {'Mild': 0, 'Moderate': 1, 'No_DR': 2, 'Proliferate_DR': 3, 'Severe': 4}


In [11]:
batch = next(train_generator)
print("Batch type:", type(batch))
print("Batch length:", len(batch))  # Should be 2 (images, labels)
print("Image batch shape:", batch[0].shape)
print("Label batch shape:", batch[1].shape)

Batch type: <class 'tuple'>
Batch length: 2
Image batch shape: (32, 224, 224, 3)
Label batch shape: (32, 5)


In [11]:
#sample batch to verify shapes
sample_imgs , sample_labels = next(train_generator)
print("Sample image shape:",sample_imgs.shape)
print("Sample labels shape:",sample_labels.shape)

Sample image shape: (32, 224, 224, 3)
Sample labels shape: (32, 5)


In [12]:
from tensorflow.keras import layers,models
#bulding the cnn model
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape = (224,224,3)),  #32 = filters,(3,3) = size of each filter, relu =  to convert negative values to zero
    layers.MaxPooling2D(2,2), #downsamples the feature maps to reduce size eg -> (120,120) => (60,60)

    layers.Conv2D(64, (3,3), activation = 'relu'),
    layers.MaxPooling2D(2,2), #(60,60) => (30,30)
    
    layers.Conv2D(128, (3,3), activation = 'relu'),
    layers.MaxPooling2D(2,2), #(15,15)

    layers.Flatten(), #converts 2D feature maps into 1d vector to be passed to fully connected layers
    
    layers.Dense(128, activation = 'relu'), # fully connected layer with 128 neurons helps to understand complex relations
    layers.Dropout(0.5), #randomly drop 50% neurons during trainig to avoid overfitting

    layers.Dense(5, activation = 'softmax') # 5 classes , softmax=> converts output into probabilities
])

model.summary()

C:\Users\shrau\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,169,605 (42.61 MB)

 Trainable params: 11,169,605 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
class_counts = [370,999,1805,295,193]

class_labels = np.concatenate([
    np.full(class_counts[i], i) for i in range(len(class_counts))
])

class_weights = compute_class_weight(class_weight = 'balanced',classes = np.arange(5), y= class_labels)

class_weights = class_weights / np.sum(class_weights)

class_weights_dict = dict(enumerate(class_weights/ np.sum(class_weights)))

print("Normalized class weights: ", class_weights_dict)

Normalized class weights:  {0: 0.21067302166543342, 1: 0.07802704506127163, 2: 0.04318505153252652, 3: 0.2642339593769843, 4: 0.40388092236378426}


In [12]:
print("Train Generator Classes:", train_generator.class_indices)
print("Class Weights Dict:", class_weights_dict)


Train Generator Classes: {'Mild': 0, 'Moderate': 1, 'No_DR': 2, 'Proliferate_DR': 3, 'Severe': 4}
Class Weights Dict: {0: 0.21067302166543342, 1: 0.07802704506127163, 2: 0.04318505153252652, 3: 0.2642339593769843, 4: 0.40388092236378426}


In [14]:
#compiling the model
model.compile(
    loss = 'categorical_crossentropy',   #categorical_crossentropy (since it’s multi-class classification)
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),  #Adam(adaptive learning rate for better convergence)
    metrics = ['accuracy'] 
)

In [15]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor = 'val_loss',
    patience=5,
    restore_best_weights = True
)

In [18]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks = [early_stopping],
    steps_per_epoch = len(train_generator),
    validation_steps = len(val_generator)
    
)

C:\Users\shrau\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - accuracy: 0.5017 - loss: 1.4522 - val_accuracy: 0.5075 - val_loss: 1.1261
Epoch 2/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 86s 934ms/step - accuracy: 0.5947 - loss: 1.1218 - val_accuracy: 0.5185 - val_loss: 1.1642
Epoch 3/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 85s 919ms/step - accuracy: 0.6001 - loss: 1.0761 - val_accuracy: 0.5732 - val_loss: 1.0745
Epoch 4/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 84s 916ms/step - accuracy: 0.6299 - loss: 1.0375 - val_accuracy: 0.5964 - val_loss: 1.0441
Epoch 5/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 85s 921ms/step - accuracy: 0.6074 - loss: 1.0503 - val_accuracy: 0.5841 - val_loss: 1.1009
Epoch 6/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 84s 917ms/step - accuracy: 0.6436 - loss: 1.0318 - val_accuracy: 0.6115 - val_loss: 1.0108
Epoch 7/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 85s 924ms/step - accuracy: 0.6518 - loss: 0.9866 - val_accuracy: 0.6361 - val_loss: 0.9809
Epoch 8/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 142s 917ms/step - accuracy: 0.6503 - loss: 0.9838 - val_accur

In [19]:
model.save("Diabetic_Retinopathy_Model.h5")

In [16]:
from tensorflow.keras.models import load_model

model = load_model("Diabetic_Retinopathy_Model.h5")
print("Model loaded successfully")

Model loaded successfully


In [17]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("Model loaded and compiled successfully!")

Model loaded and compiled successfully!


In [18]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = "ProliferateTest.png"
img = image.load_img(img_path,target_size =(224,224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array,axis  = 0) /255.0

#predictions
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)
print("Predicted class:",predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
Predicted class: 1
